In [2]:
import pandas as pd
def get_dir_filenames(path,ftype):
    '''获取目录下所有指定格式的文件名'''
    import os
    names = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if ftype in file:
                names.append(file)
    return names

In [ ]:
import pandas as pd
from time import sleep
import re
import threading
import random
from selenium import webdriver # 从selenium导入webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import os
from itertools import product
import requests
from bs4 import BeautifulSoup

index_url = 'http://www.sinobook.com.cn/'
options = webdriver.ChromeOptions()
with open('succeed.txt','r',encoding='utf-8') as f:
    data = f.readlines()
succeed = [x.strip() for x in data] #记录完成的名单

def get_info(colleges):
    global succeed
    random.shuffle(colleges)
    user_agent_list = [ \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60" \
        "Opera/8.0 (Windows NT 5.1; U; en)"\
        "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0" \
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER" \
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0" \
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)" \
    ]
    for college in colleges:
        if college in succeed:
            print(college + ' 之前已下载')
            continue
#         try:
        UserAgent = 'user-agent=' + random.choice(user_agent_list)
        options.add_argument(UserAgent)
        #index_path = 'D:\\codes\\GitProject\\工作区\\首经贸数据采集项目\\教材数据\\data'
        #prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': index_path}  #下载路径
        #options.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(chrome_options=options)
        driver.get(index_url)
        sleep(1)
        input_btn = driver.find_element_by_id('sKeyword')
        input_btn.send_keys(college)
        driver.find_elements_by_class_name('sehbtn1')[4].click() #按出版社查询
        sleep(2)
        driver.switch_to_window(driver.window_handles[1]) #可能不是这个写法

        #news = driver.find_element_by_class_name('tblBrow').find_elements_by_tag_name('a')
        temp_list = []
        while True: #如果有下一页
            try:
                table = driver.find_element_by_id('ContentArea').find_elements_by_tag_name('tr')[1:-1] #第一个为表抬头
            except:
                print(college + ' 没有检索结果')
                succeed.append(college)
                driver.quit()
                break
            have_next = True
            have_next = True
            for record in table:
                '''书名，出版社，ISBN，作者，印次，出版日期，定价'''
                items  = record.find_elements_by_tag_name('td')[1:] #第一个是无用的标签
                book_name = items[0].text
                publish = items[1].text
                ISBN = items[2].text
                author = items[3].text
                type_counts = items[4].text
                date = items[5].text
                if len(date) > 0:
                    if int(date[:4]) < 2016:
                        have_next = False
                        break
                price = items[6].text
                row = {'书名':book_name,'出版社':publish,'ISBN':ISBN,'作者':author,'印次':type_counts,'出版日期':date,'定价':price}
                temp_list.append(row)
            if not have_next:
                break
            try:
                driver.find_element_by_link_text('下一页>').click() #下一页
                driver.switch_to_window(driver.window_handles[1])
                driver.close()
                driver.switch_to_window(driver.window_handles[1])
                sleep(4)
            except:
                print('没有下一页')
                break
        df = pd.DataFrame(temp_list)
        df.to_excel('data/'+college+'-教材信息.xlsx')
        print(college+' 已经下载完毕')
        driver.quit()
        succeed.append(college)
        succeed = list(set(succeed))
        with open('succeed.txt','w',encoding='utf-8') as f:
            for col in succeed:
                f.write(col)
                f.write('\n')
        if len(set(succeed)) == 27:
            print('教材信息全部下载完毕')
            break
#         except:
#             driver.quit()
#             print(college+' 下载出现异常')
            
def to_download(counts):
    data = pd.read_excel('高校名单.xlsx',sheet_name='全部高校')
    colleges = data['高校名称']
    for i in range(counts):
        print('第'+str(i+1)+'次循环')
        get_info(colleges)
    
get_info(['清华大学'])
#if __name__ == '__main__':
#     counts = 2
#     threads = []
#     for i in range(1):
#         t = threading.Thread(target=to_download,args=(counts,))
#         threads.append(t)
#     for t in threads:
#         t.start()
#     for thread in threads:
#         thread.join()
        

### 教材类别划分

In [ ]:
经济学', '统计学', '工商管理

经济学 ['经济', '保险', '资本', '汇率', '货币', '财务', '税', '外汇', '贸易', '外资', '股票', '投资', '金融', '证券', '筹资', '基金', '股权', '银行', '信托', '债券', '财政', '货币', '银行', '企业管理', '财务', '会计', '市场', '成本', '利润', '市场规模', '城市化', '城镇化', '业绩', '支出', '效益', '通货膨胀', '投入产出', '进口', '出口', '民营', '绩效', '产能', '补贴', '失业', '房价', '物价', '信贷']
统计学 ['统计', '样本', '测量', '尺度', '相关性', '相关系数', '数理', '国势', '政治算术']
工商管理 temp = ['市场', '工商管理', '工商', '管理', '上市', '公司', '注册会计师', '内部控制', '财政管理', '公司治理', '审计', '生产管理', '生产控制', '生产组织', '生产计划', '经营', '风险投资', '技术创新', '股权结构', '市场战略', '组织价值观', '市场导向', '质量', '绩效', '组织变革', '盈余', '股东控制', '财务', '投资', '企业控制', '筹资', '利润', '经济管理', '管理成本', '成本管理', '研发管理', '团队建设', '流程设计', '风险', '品牌', '人力资源', '人力管理', '绩效', '劳动关系', '福利管理', '管理福利', '会计学', '企业战略', '资源规划']


In [ ]:
import pandas as pd
jjx = ['经济', '保险', '资本', '汇率', '货币', '财务', '税', '外汇', '贸易', '外资', '股票', '投资', '金融', '证券', '筹资', '基金', '股权', '银行', '信托', '债券', '财政', '货币', '银行', '企业管理', '财务', '会计', '市场', '成本', '利润', '市场规模', '城市化', '城镇化', '业绩', '支出', '效益', '通货膨胀', '投入产出', '进口', '出口', '民营', '绩效', '产能', '补贴', '失业', '房价', '物价', '信贷']
tjx = ['统计', '样本', '测量', '尺度', '相关性', '相关系数', '数理', '国势', '政治算术']
gsgl = ['市场', '工商管理', '工商', '管理', '上市', '公司', '注册会计师', '内部控制', '财政管理', '公司治理', '审计', '生产管理', '生产控制', '生产组织', '生产计划', '经营', '风险投资', '技术创新', '股权结构', '市场战略', '组织价值观', '市场导向', '质量', '绩效', '组织变革', '盈余', '股东控制', '财务', '投资', '企业控制', '筹资', '利润', '经济管理', '管理成本', '成本', '研发管理', '团队建设', '流程设计', '风险', '品牌', '人力资源', '人力', '绩效', '劳动', '福利管理', '管理福利', '会计', '企业战略', '资源规划']



def is_in(kws,text):
    for item in kws:
        if item in text:
            return True
    return False

In [9]:
fnames = get_dir_filenames('data','.xlsx')
data = pd.read_excel('高校名单.xlsx',sheet_name='管理学')
college = data['高校名称']
college = [x for x in college]

for name in fnames:
    if name.split('-')[0] in college:
        data = pd.read_excel('data/'+name)
        temp_list = []
        for index, row in data.iterrows():
            if is_in(gsgl,row['书名']):
                temp_list.append(row)
        df = pd.DataFrame(temp_list)
        df.to_excel('data/工商管理/'+name)

### 提取2016 - 2019

In [16]:
data_path = 'data/工商管理'
fnames = get_dir_filenames(data_path,'.xlsx')
for name in fnames:
    data = pd.read_excel(data_path+'/'+name)
    temp_list = []
    for index,row in data.iterrows():
        if type(row['出版日期']) != float:
            if int(row['出版日期'][:4]) > 2015:
                temp_list.append(row)
    data = pd.DataFrame(temp_list)
    data.to_excel(data_path+'/'+name)

### 统计表

In [19]:
data_path = 'data/统计学'
fnames = get_dir_filenames(data_path,'.xlsx')
temp_list = []
for name in fnames:
    data = pd.read_excel(data_path+'/'+name)
    row = {'学科':'统计学','学校':name.split('-')[0],'教材数':len(data)}
    temp_list.append(row)
df = pd.DataFrame(temp_list)
df.to_excel('data/统计学-教材数-统计表.xlsx')

### 教材数据融合

In [3]:
fnames = get_dir_filenames('原始数据','.xlsx')
temp = []
for name in fnames:
    df = pd.read_excel('原始数据/'+name)
    temp.append(df)
df = pd.concat(temp)
df.to_excel('教材数据融合.xlsx')
len(df)

29336